## 数据结构和算法
### 1 将序列分解为单独的变量
#### 1-1 问题
我们有一个包含N个元素的元组或序列，现在想将它分解为N个单独的变量。
#### 1-2 解决方案
任何序列(或可迭代的对象)都可以通过一个简单的复制操作来分解为单独的变量。唯一的要求就是变量的总数和结构要与序列相吻合。例如:

In [1]:
p = (4,5)
x, y = p
x

4

In [2]:
y

5

In [3]:
data = ['ACME', 50, 91.1, (2012, 12, 21)]
name, shares, price, date = data
name

'ACME'

In [4]:
date

(2012, 12, 21)

In [5]:
name, shares, price, (year, mon, day) = data
name

'ACME'

In [6]:
year

2012

如果元素的数量不匹配，将得到一个错误的提示。例如:

In [7]:
p = (4,5)
x, y, z = p
x

ValueError: not enough values to unpack (expected 3, got 2)

#### 1-3 讨论

实际上不仅仅只是元组或列表，只要对象恰好是可迭代的，那么就可以执行分解操作。这包括字符串，文件，迭代器以及生成器。比如:

In [8]:
s = 'hello'
a, b, c, d, e = s
a

'h'

In [9]:
e

'o'

当做分解操作时，有时候可能想丢弃某些特定的值。python并没有提供特殊的语法来实现这一点，但是通常可以选一个用不到的变量名，以此来作为丢弃的值的名称。例如:

In [10]:
data = ['ACME', 50, 90.1, (2012, 12, 21)]
_, shares, price, _ = data
shares

50

In [11]:
price

90.1

### 2 从任意长度的可迭代对象中分解元素
#### 2-1 问题
需要从某个可迭代对象中分解出N个元素，但是这个可迭代对象的长度可能超过N，这会导致出现“分解的值过多”的异常。
#### 2-2 解决方案
Python的“*表达式”可以用来解决这个问题。例如，假设开设了一门课程，并决定在期末作业成绩中去掉第一个和最后一个，只对中间剩下的成绩做平均统计。如果只有4个成绩，也许可以将4个都分解出来，但是如果有24个呢？“*表达式”使这一切变得简单:

In [12]:
def drop_first_last(grades):
    first, *middle, last = grades
    return avg(middle)

另一个用例是假设有一些用户记录，记录由姓名和电子邮件地址组成，后面跟着任意数量的电话号码。则可以像这样分解记录:

In [13]:
user_record = ('Dave', 'dave@example.com', '773-555-1212', '847-555-1212')
name, email, *phone_numbers = user_record
name

'Dave'

In [14]:
email

'dave@example.com'

In [15]:
phone_numbers

['773-555-1212', '847-555-1212']

由*修饰的变量也可以位于列表的第一个位置。例如，比方说用一系列的值，来代表公司过去8个季度的销售额。如果相对最近一个季度的销售额同前7个季度的平均值作比较，可以这么做:

In [16]:
*trailing, current = [10,8,8,8,8,8,7]
trailing

[10, 8, 8, 8, 8, 8]

In [17]:
current

7

#### 2-3 讨论
对于分解未知或任意长度的可迭代对象，这种扩展的分解操作可谓是量身定制的工具。通常，这类可迭代对象中会有一些已知的组件或模式(例如，元素1之后的所有内容都是电话号码)。
*式语法在迭代一个变长的元组序列时尤其有用。例如，假设有一个带标记的元组序列:

In [18]:
records = [
    ('foo', 1, 2),
    ('bar','hello'),
    ('foo', 3, 4),
    ]

def do_foo(x,y):
    print('foo',x,y)
def do_bar(s):
    print('bar',s)
    
for tag, *args in records:
    if tag == 'foo':
        do_foo(*args)
    elif tag == 'bar':
        do_bar(*args)

foo 1 2
bar hello
foo 3 4


当和某些特定的字符串处理操作箱结合，比如做拆分（splitting）操作时，这种*式的语法所支持的分解操作也非常有用。例如:

In [19]:
line = 'nobody:*:-2:-2:Unpeivileged User:/var/empty:/user/bin/false'
uname, *fields, homedir, sh = line.split(':')
uname

'nobody'

In [20]:
homedir

'/var/empty'

In [21]:
sh

'/user/bin/false'

有时候可能想分解出某些值然后丢弃它们。在分解的时候，不能只是指定一个单独的*，但是可以使用几个常用的来表示待丢弃的变量名，比如_或者ign(ignored)。例如:

In [22]:
record = ('ACME', 50, 123.45, (12, 18, 2012))
name, *_, (*_, year) = record
name

'ACME'

In [23]:
year

2012

*分解操作和各种函数式语言中的列表处理功能有着一定的相似性。例如，如果有一个列表，可以像下面这样轻松将其分解为头部和尾部:

In [24]:
items = [1, 10, 7, 4, 5, 9]
head, *tail = items
head

1

In [25]:
tail

[10, 7, 4, 5, 9]

在编写执行这类拆分功能的函数时，人们可以假设这是为了实现某种精巧的递归算法。例如:

In [26]:
def sum(items):
    head, *tail = items
    return head + sum(tail) if tail else head

sum(items)

36

### 3保存最后N个元素
#### 3-1 问题
我们希望在迭代或是其他形式的处理过程中对最后几项记录做一个有限的历史记录。
#### 3-2 解决方案
保存有限的历史记录可算是collections.deque的完美应用场景了。例如，下面的代码对一系列文本行做简单的文本匹配操作，当发现有匹配时就输出当前的匹配行以及最后检查过的N行文本。

In [27]:
from collections import deque

def search(lines, pattern, history=5):
    previous_lines = deque(maxlen=history)
    for line in lines:
        if pattern in line:
            yield line, previous_lines
        previous_lines.append(line)
        
# Example use on a file
if __name__ == '__main__':
    with open('somefile.txt') as f:
        for line, previous in search(f, 'python', 5):
            for pline in previous:
                print(pline, end='')
            print(line, end='')
            print('-'*20)

python
--------------------
python
eeepython
--------------------
python
eeepython
ddpython
--------------------
python
eeepython
ddpython
pffython
dddpython
--------------------
ddpython
pffython
dddpython
ffptyfh
dfh
python1
--------------------
pffython
dddpython
ffptyfh
dfh
python1
python2--------------------


#### 3-3 讨论
如同上面的代码片段中所做的一样，当编写搜索某项记录的代码时，通常会用到含有yield关键字的生成器函数。这将处理搜索过程的代码和使用搜索结果的代码成功解耦开来。
deque(maxlen=N)创建了一个固定长度的队列。当有新纪录加入而队列已满时会自动移除最老的那条记录。例如:

In [28]:
q = deque(maxlen=3)
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [29]:
q.append(4)
q

deque([2, 3, 4])

In [30]:
q.append(5)
q

deque([3, 4, 5])

更普遍的是，当需要一个简单的队列结构时，deque可助你一臂之力。如果不指定队列的大小，也就得到无界限的队列，可以再两端执行添加和弹出操作，例如:

In [31]:
q = deque()
q.append(1)
q.append(2)
q.append(3)
q

deque([1, 2, 3])

In [32]:
q.appendleft(4)
q

deque([4, 1, 2, 3])

In [33]:
q.pop()

3

In [34]:
q

deque([4, 1, 2])

In [35]:
q.popleft()

4

从队列两端添加或弹出元素的复杂度都是O(1)。这和列表不同，当从列表的头部插入或移除元素时，列表的复杂度为O(n)。

### 4 找到最大或最小的N个元素
#### 4-1 问题
我们想在某个集合中找出最大或最小的N个元素。
#### 4-2 解决方案
heapq模块中有两个函数,nlargest()和nsmallest()，它们正是我们所需要的。例如:

In [ ]:
import heapq

nums = [1, 8, 2, 23, 7 ,-4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3,nums))

In [ ]:
print(heapq.nsmallest(3,nums))

这两个函数都可以接收一个参数key，从而允许它们工作在更加复杂的数据结构之上。例如:

In [ ]:
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price':91.1},
    {'name': 'AAPL', 'shares': 50, 'price':543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares':75, 'price': 115.65}
    ]
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])
cheap

In [ ]:
expensive

#### 4-3 讨论
如果正在寻找最大或最小的N个元素，且同集合中元素的总数目项目，N很小，纳闷下面这些函数可以提供更好的性能。这些函数首先会在底层将数据转化成列表，且元素会以堆的顺序排列。例如:

In [ ]:
nums = [1,8,2,23,7,-4,18,23,42,37,2]
import heapq
heap = list(nums)
heapq.heapify(heap)
heap

堆最重要的特性就是heap[0]总是最小的那个元素。此外，接下来的元素可依次通过heapq.heappop()方法轻松找到。该方法会将第一个元素弹出，然后以第二小的元素取代之。例如要找到第3小的元素，可以这样做:

In [ ]:
heapq.heappop(heap)

In [ ]:
heapq.heappop(heap)

In [ ]:
heapq.heappop(heap)

当所要找的元素数量相对较少，函数nlargest()和nsamllest()才是最合适的。如果只是简单地想要找到最小或最大的元素(N=1)，那么用min()和max()会更加快。同样，如果N和集合本身的大小差不多，通常更快的方法是对集合排序，然后做切片操作。

### 5 实现优先级队列
#### 5-1 问题
我们想要实现一个队列，ta能够以给定的优先级来对元素排序，且每次pop操作时都会返回优先级最高的那个元素。
#### 5-2 解决方案
下面的类利用heapq模块实现了一个简单的优先级队列:

In [36]:
import heapq
class PriorityQueue:
    def __init__(self):
        self._queue = []
        self._index = 0
        
    def push(self, item, priority):
        heapq.heappush(self._queue, (-priority, self._index, item))
        self._index += 1
    def pop(self):
        return heapq.heappop(self._queue)[-1]

下面是如何使用这个类的例子:

In [39]:
class Item:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return 'Item({!r})'.format(self.name)
    
q = PriorityQueue()
q.push(Item('foo'),1)
q.push(Item('bar'),5)
q.push(Item('spam'),4)
q.push(Item('grok'),1)
q.pop()

Item('bar')

In [40]:
q.pop()

Item('spam')

In [41]:
q.pop()

Item('foo')

In [42]:
q.pop()

Item('grok')

我们观察到第一次执行pop()操作时返回的元素具有最高的优先级。我们也观察到拥有相同优先级的两个元素（foo和grok）返回的顺序同它们插入到队列的顺序相同。

#### 5-3 讨论
上面的代码片段核心在于heapq模块的使用。函数heapq.heappush()以及heapq.heappop()分别实现将元素从列表_queue中插入和移除，且保证列表中的第一个元素的优先级最低。在这段代码中，队列以元组(-priority,index,item)的形式组成。把priority取负值是为了让队列能够按元素的优先级从高到低的顺序排列。变量index的作用是为了将具有相同优先级的元素以适当的顺序来排列。
为了说明Item实例是没法进行次序比较的，我们来看下面这个例子:

In [43]:
a = Item('foo')
b = Item('bar')
a<b

TypeError: unorderable types: Item() < Item()

如果以元组(priority,item)的形式来表示元素，那么只要优先级不同，它们就可以进行比较。但是，如果两个元组的优先级相同，作比较操作还是会失败。例如:

In [44]:
a = (1, Item('foo'))
b = (5, Item('bar'))
a<b

True

In [45]:
c = (1, Item('grok'))
a<c

TypeError: unorderable types: Item() < Item()

通过引入额外的索引值，以(prioroty,index,item)的方式建立元组，就可以完全避免这个问题。因为没有哪两个元组会有相同的索引值。

In [46]:
a = (1, 0,Item('foo'))
b = (5, 1,Item('bar'))
c = (1, 2,Item('grok'))
a<b

True

In [47]:
a<c

True

### 6 在字典中将键映射到多个值上
#### 6-1 问题
我们想要一个能将键(key)映射到多个值得字典(即所谓的一键多值字典)
#### 6-2 解决方案
字典是一种关联容器，每个键都映射到一个单独的值上。如果想要让键值映射到多个值，需要将这多个值保存到另一个容器如列表或集合中。例如，可能会像这样创建字典:

In [49]:
d = {
    'a':[1,2,3],
    'b':[4,5]
    }
e = {
    'a':{1,2,3},
    'b':{4,5} # 集合
    }

为了能方便的创建这样的字典，可以利用collections模块中的defaultdict类。defaultdict的一个特点就是它会自动初始化第一个值，这样只需要关注添加元素即可。例如:

In [52]:
from collections import defaultdict

d = defaultdict(list)
d['a'].append(1)
d['a'].append(2)
d

defaultdict(list, {'a': [1, 2]})

In [55]:
d = defaultdict(set)
d['a'].add(1)
d['a'].add(2)
d

defaultdict(set, {'a': {1, 2}})

#### 6-3 讨论
原则上，构建一个一键多值字典是很容易的。但是如果试着自己对第一个值做初始化操作，这就会变得很复杂：

In [ ]:
d = {}
for key,value in pairs:
    if key not in d:
        d[key] = []
    d[key].append(value)

使用defaultdict后代码会清晰得多:

In [57]:
d = defaultdict(list)
for key,value in pairs:
    d[key].append(value)

NameError: name 'pairs' is not defined

### 7 让字典保持有序
#### 7-1 问题
我们想要创建一个字典，同时当对字典做迭代或序列化操作时，也能控制其中元素的顺序。
#### 7-2 解决方案
要控制字典中元素的顺序，可以使用collections模块中的OrderdDict类。当对字典做迭代时，它会严格按照元素初始添加的顺讯进行。例如:

In [59]:
from collections import OrderedDict
d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4

# Outputs "foo 1","bar 2",,,
for key in d:
    print(key,d[key])

foo 1
bar 2
spam 3
grok 4


当想要构建一个映射结构以便稍后对其做序列化或编码成另一种格式时，OrderedDict就显得特别有用。例如，如果想在进行JSON编码时精确控制各字段的顺序，那么只要首先在OrderedDict中构建数据就行了。

In [62]:
import json
json.dumps(d)

'{"foo": 1, "bar": 2, "spam": 3, "grok": 4}'

OrderedDict内部维护了一个双向链表，它会根据元素加入的顺序来排列键的位置。第一个新加入的元素被放置在链表的末尾。接下来对已存在的键做重新赋值不会改变键的顺序。
请注意OrderedDict的大小是普通字典的两倍多，这是由于它额外创建的链表所致。因此，如果打算构建一个涉及大量OrderedDict实例的数据结构(例如从CSV文件中读取100000行内容到OrderedDict列表中)，那么需要认真对应用做需求分析，从而判断使用OrderedDict所带来的好处是否能超越因额外内存开销所带来的缺点。

### 8 与字典有关的计算问题
#### 8-1 问题
我们想在字典上对数据执行各式各样的计算(比如求最小值，最大值，排序等)
#### 8-2 解决方案
假设有一个字典在股票名称和对应的价格间做了映射:

In [63]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
    }


为了能对字典内容做些有用的计算，通常会利用zip()将字典的键和值反转过来。例如，下面的代码会告诉我们如何找出价格最低和最高的股票。

In [65]:
min_price = min(zip(prices.values(),prices.keys()))
min_price

(10.75, 'FB')

In [66]:
max_price = max(zip(prices.values(),prices.keys()))
max_price

(612.78, 'AAPL')

同样，要对数据排序只要使用zip()再配合sorted()就可以了，比如：

In [68]:
prices_sorted = sorted(zip(prices.values(),prices.keys()),reverse=True)
prices_sorted

[(612.78, 'AAPL'),
 (205.55, 'IBM'),
 (45.23, 'ACME'),
 (37.2, 'HPQ'),
 (10.75, 'FB')]

当进行这些计算时，请注意zip()创建了一个迭代器，它的内容只能被消费一次。例如下面的代码就是错误的:

In [69]:
prices_and_names = zip(prices.values(),prices.keys())
print(min(prices_and_names)) # OK
print(max(prices_and_names)) # ValueErorr: max() arg is an empty sequence

(10.75, 'FB')


ValueError: max() arg is an empty sequence

#### 8-3 讨论
如果尝试在字典上执行常见的数据操作，将会发现它们只会处理键，而不是值。例如:

In [70]:
min(prices)

'AAPL'

In [71]:
max(prices)

'IBM'

这可能不是我们所期望的，因为实际上我们是尝试对字典的值做计算。可以利用字典的values（）方法来解决这个问题:

In [72]:
min(prices.values())

10.75

In [73]:
max(prices.values())

612.78

不幸的是，通常这也不是我们所期望的。比如，我们可能想知道相应的键所关联的信息是什么(例如那支股票的价格最低？)
如果提供一个key参数传递给min()和max()，就能得到最大值和最小值所对应的键是什么。例如:

In [75]:
min(prices,key=lambda k: prices[k])

'FB'

In [76]:
max(prices,key=lambda k: prices[k])

'AAPL'

但是如果要得到最小值的话，还需要额外执行一次查找。例如:

In [78]:
min_value = prices[min(prices,key=lambda k:prices[k])]
min_value

10.75

利用zip()的解决方案是通过将字典的键-值对"反转"为值-键对序列来解决这个问题的。
当在这样的元组上执行比较操作时，值会先进行比较，然后才是键。这完全符合我们的期望，允许我们用一条单独的语句轻松地对字典里的内容做整理和排序。
应该要注意的是，当涉及(value,key)作比较时，如果碰巧有多个条目拥有相同的value值，那么此时key将用来作为判定结果的依据。例如，在计算min()和max()时，如果碰巧value的值相同，则将返回拥有最小或最大key值得那个条目:

In [79]:
prices = {'AAA':45.23,'ZZZ':45.23}
min(zip(prices.values(),prices.keys()))

(45.23, 'AAA')

In [80]:
max(zip(prices.values(),prices.keys()))

(45.23, 'ZZZ')

### 9 在两个字典中寻找相同点
#### 9-1 问题
有两个字典，我们想找出它们中间可能相同的地方(想听的键，相同的值等)
#### 9-2 解决方案
考虑如下两个字典:

In [81]:
a = {
    'x':1,
    'y':2,
    'z':3
    }
b = {
    'w':10,
    'x':11,
    'y':2
    }

要找出这两个字典中相同之处，只需要通过keys()或者items（）方法执行常见的集合操作即可。例如:

In [82]:
# find keys in common
a.keys() & b.keys()

{'x', 'y'}

In [83]:
# find keys in a that are not in b
a.keys() - b.keys()

{'z'}

In [85]:
# find (key,value) pairs in common
a.items() & b.items()

{('y', 2)}

这些类型的操作也可用来修改或过滤字典中的内容。例如，假设想创建一个新的字典，其中会去掉某些键。下面是使用了字典推导式的代码示例:

In [87]:
# make a new dictionary with certain key removed
c = {key:a[key] for key in a.keys() - {'z','w'}}
c

{'x': 1, 'y': 2}

#### 9-3 讨论
字典就是一系列键和值之间的映射集合。字典的keys()方法会返回keys-viem对象，其中暴露了所有的键。关于字典的键有一个很少人知道的特性，那就是它们也支持常见的集合操作，比如求并集、交集和差集。
字典的items()方法返回由(key,value)对组成的items-view对象。这个对象支持类似的集合操作，可用来完成找出两个字典间有哪些键值对有相同之处的操作。
尽管相似，但字典的values()方法并不支持集合操作。部分原因是因为在字典中键和值是不同的，从值得角度来看并不能保证所有的值都是唯一的。

### 10 从序列中移除重复项且保持元素间顺序不变
#### 10-1 问题
我们想去除序列中出现的重复元素，但仍然保持剩下的元素顺序不变。
#### 10-2 解决方案
如果序列中的值是可哈希（hashable）的，那么这个问题可以通过使用集合和生成器轻松解决。示例如下:

In [89]:
def dedupe(items):
    seen =set()
    for item in items:
        if item not in seen:
            yield item
            seen.add(item)

这里是如何使用这个函数的例子:

In [90]:
a = [1,5,2,1,9,1,5,10]
list(dedupe(a))

[1, 5, 2, 9, 10]

只有当序列中的元素是可哈希的时候才可以这么做。如果想在不可哈希的对象(比如列表)序列中去除重复项，需要对上述代码稍作修改:

In [92]:
def dedupe(items,key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)

这里参数key的作用是指定一个函数用来将序列中的元素转化为可哈希的类型，这么做的目的是为了检测重复项，它可以像这样工作:

In [101]:
a = [{'x':1,'y':2},{'x':1,'y':3},{'x':1,'y':2},{'x':2,'y':4}]
list(dedupe(a,key=lambda d:(d['x'],d['y'])))

[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]

In [102]:
list(dedupe(a,key=lambda d: d['x']))

[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]

如果希望在一个较复杂的数据结构中，只根据对象的某个字段或属性来去除重复项，那么后一种解决方法同样能完美解决。

#### 10-3 讨论
如果想要做的只是去除重复项，那么通常足够简单的办法就是构建一个集合。例如:

In [105]:
a = [1,5,2,1,9,1,5,10]

In [106]:
set(a)

{1, 2, 5, 9, 10}

但是这种方法不能保证元素间的顺序不变，因此得到的结果会被打乱。前面展示的解决方案可以避免这个问题。
本节中对生成器的使用反映出一个事实，那就是我们可能会希望这个函数尽可能的通用，不必绑定在只能对列表进行处理。比如，想读一个文件，去除其中重复的文本行，可以只需这样处理:

In [109]:
with open('somefile.txt','r') as f:
    for line in dedupe(f):
        pass

### 11 对切片命名
#### 11-1 问题
我们的代码已经变得无法阅读，到处都是硬编码的切片索引，我们想将它们清理干净。
#### 11-2 解决方案
假设有一些代码用来从字符串的固定位置中取出具体的数据:

In [113]:
record = '....................100............513...'
cost = int(record[20:23])*10
cost

1000

与其这样做，为什么不对切片命名呢？

In [114]:
SHARES = slice(20,23)
int(record[SHARES])*10

1000

在后一种版本中，由于避免了使用许多神秘难懂的硬编码索引，我们的代码就变得清晰许多。

#### 11-3 讨论
作为一条基本准则，代码中如果有很多硬编码的索引值，将导致可读性和可维护性都不佳。例如如果一年以后再回过头来看代码，你会发现自己很想知道当初编写这些代码时自己在想些什么。前面展示的代码可以让我们对代码的功能有着更加清晰得认识。
一般来说，内置的slice()函数会创建一个切片对象，可以用在任何允许进行切片操作的地方，例如:

In [115]:
items = [0,1,2,3,4,5,6]
a = slice(2,4)
items[2:4]

[2, 3]

In [116]:
items[a]

[2, 3]

In [118]:
items[a] = [10,11]
items

[0, 1, 10, 11, 4, 5, 6]

In [119]:
del items[a]
items

[0, 1, 4, 5, 6]

假如有一个slice对象的实例s，可以分别通过s.start,s.stop,s.step属性来得到关于该对象的信息。例如:

In [120]:
s = slice(10,50,2)
s.start

10

In [121]:
s.stop

50

In [122]:
s.step

2

此外，可以通过使用indices(size)方法将切片映射到特定大小的序列上。这会返回一个(start,stop,step)元组，所有的值都已经恰当地限制在边界以内(当做索引操作时可避免出现IndexError异常)。例如:

In [127]:
s = 'HelloWorld'
a.indices(len(s))

(2, 4, 1)

In [128]:
for i in range(*a.indices(len(s))):
    print(s[i])

l
l


### 12 找出序列中出现次数最多的元素
#### 12-1 问题
我们有一个元素序列，想知道在序列中出现次数最多的元素是什么。
#### 12-2 解决方案
collections模块中的Counter类正是为此类问题所设计的。它甚至有一个非常方便的most_common()方法可以直接告诉我们。
为了说明用法，假设有一个列表，列表中是一系列单词，我们想找出哪些单词出现最为频繁：

In [130]:
words = [
    'look','into','my','eyes','look','into','my','eyes',
    'the','eyes','the','eyes','the','eyes','not','around','the',
    'eyes',"don't",'look','around','the','eyes','look','into',
    'my','eyes',"you're",'under'
    ]
from collections import Counter
word_counts = Counter(words)
top_three = word_counts.most_common(3)
print(top_three)

[('eyes', 8), ('the', 5), ('look', 4)]


#### 12-3 讨论
可以给Counter对象提供任何可哈希的对象序列作为输入。早底层实现中，Counter是一个字典，在元素和它们出现的次数间做了映射。例如:

In [131]:
word_counts['not']

1

In [132]:
word_counts['eyes']

8

如果想手动增加计数，只需简单地自增即可:

In [133]:
morewords = ['why','are','you','not','looking','in','my','eyes']
for word in morewords:
    word_counts[word]+=1

In [135]:
word_counts['eyes']

9

另一种方式是使用update()方法

In [136]:
word_counts.update(morewords)

关于Counter对象有一个不为人知的特性，那就是它们可以轻松地同各种数学运算操作结合起来使用。例如:

In [137]:
a = Counter(words)
b = Counter(morewords)
a

Counter({'around': 2,
         "don't": 1,
         'eyes': 8,
         'into': 3,
         'look': 4,
         'my': 3,
         'not': 1,
         'the': 5,
         'under': 1,
         "you're": 1})

In [138]:
b

Counter({'are': 1,
         'eyes': 1,
         'in': 1,
         'looking': 1,
         'my': 1,
         'not': 1,
         'why': 1,
         'you': 1})

In [140]:
# combine counts
c = a + b
c

Counter({'are': 1,
         'around': 2,
         "don't": 1,
         'eyes': 9,
         'in': 1,
         'into': 3,
         'look': 4,
         'looking': 1,
         'my': 4,
         'not': 2,
         'the': 5,
         'under': 1,
         'why': 1,
         'you': 1,
         "you're": 1})

In [144]:
# subtract counts
d = a - b
d

Counter({'around': 2,
         "don't": 1,
         'eyes': 7,
         'into': 3,
         'look': 4,
         'my': 2,
         'the': 5,
         'under': 1,
         "you're": 1})

不用说，当面对任何需要对数据制表或计数的问题时，Counter对象都是你手边的得力工具。

### 13 通过公共键对字典列表排序
#### 13-1 问题
我们有一个字典列表，想根据一个或多个字典中的值来对列表排序。
#### 13-2 解决方案
利用operator模块中的itemgetter函数对这类结构进行排序是非常简单的。假设通过查询数据库表项获取网站上的成员列表，我们得到了如下的数据结构:

In [146]:
rows = [
    {'fname':'Brian','lname':'Jones','uid':1003},
    {'fname':'David','lname':'Beazley','uid':1002},
    {'fname':'John','lname':'Cleese','uid':1001},
    {'fname':'Big','lname':'Jones','uid':1004},
    ]

根据所有字典中共有的字段来对这些记录排序是非常简单的，实例如下:

In [153]:
from operator import itemgetter
rows_by_fname = sorted(rows,key=itemgetter('fname'))
rows_by_uid = sorted(rows,key = lambda rows:rows['uid'])
print(rows_by_fname)
print()
print(rows_by_uid)

[{'lname': 'Jones', 'uid': 1004, 'fname': 'Big'}, {'lname': 'Jones', 'uid': 1003, 'fname': 'Brian'}, {'lname': 'Beazley', 'uid': 1002, 'fname': 'David'}, {'lname': 'Cleese', 'uid': 1001, 'fname': 'John'}]

[{'lname': 'Cleese', 'uid': 1001, 'fname': 'John'}, {'lname': 'Beazley', 'uid': 1002, 'fname': 'David'}, {'lname': 'Jones', 'uid': 1003, 'fname': 'Brian'}, {'lname': 'Jones', 'uid': 1004, 'fname': 'Big'}]


In [148]:
rows_by_lfname = sorted(rows,key=itemgetter('lname','fname'))
print(rows_by_lfname)

[{'lname': 'Beazley', 'uid': 1002, 'fname': 'David'}, {'lname': 'Cleese', 'uid': 1001, 'fname': 'John'}, {'lname': 'Jones', 'uid': 1004, 'fname': 'Big'}, {'lname': 'Jones', 'uid': 1003, 'fname': 'Brian'}]


#### 13-3讨论
在这个例子中，rows被传递给內建的sorted()函数，该函数接受一个关键字参数key。这个参数应该代表一个可调用对象(callable)，该对象从rows中接受一个单独的元素作为输入并返回一个用来做排序依据的值。itemgetter()函数创建的就是这样一个可调用对象。
函数operator.itemgetter()接受的参数可作为查询的标记，用来从rows的记录中提取出所需要的值。它可以是字典的键名称，用数字表示的列表元素或是任何可以传给对象__getitem__()方法的值。如果传多个标记给itemgetter()，那么它产生的可调用对象将返回一个包含所有元素在内的元组，然后sorted()将根据对元组的排序结果来排列输出结果，有时候会用lambda表达式来取代itemgetter()的功能。例如:

In [154]:
rows_by_fname = sorted(rows,key=lambda r:r['fname'])
rows_by_fname = sorted(rows,key=lambda r:(r['lname'],r['fname']))

这种解决方案通常也能正常工作。但是用itemgetter()通常会运行得更快些。因此如果需要考虑性能问题的话，应该使用Itemgetter()。
最后不要忘了本节中所展示的技术同样适用于min()和max()这样的函数。例如:

In [155]:
min(rows,key=itemgetter('uid'))

{'fname': 'John', 'lname': 'Cleese', 'uid': 1001}

### 14 对不原生支持比较操作的对象排序
#### 14-1 问题
我们想在同一个类的实例之间做排序，但是它们并不原生支持比较操作。
#### 14-2 解决方案
內建的sorted()函数可接受一个用来传递可调用对象(callable)的参数key，而该可调用对象会返回待排序对象中的某些值，sorted则利用这些值来比较对象。例如，如果应用中有一系列的User对象实例，而我们想通过user_id属性来对它们排序，则可以提供一个可调用对象User实例作为输入然后返回User_id。实例如下:

In [156]:
class User:
    def __init__(self,user_id):
        self.user_id = user_id
    def __repr__(self):
        return 'User({})'.format(self.user_id)
    
users = [User(23),User(3),User(99)]
users

[User(23), User(3), User(99)]

In [157]:
sorted(users,key=lambda u:u.user_id)

[User(3), User(23), User(99)]

除了可以用lambda表达式外，另一种方式是使用operator.attrgetter()。

In [158]:
from operator import attrgetter
sorted(users,key=attrgetter('user_id'))

[User(3), User(23), User(99)]

#### 14-3 讨论
要使用lambda表达式还是attrgetter()或许只是一种个人喜好，但attrgetter()要更快一些，而且具有同时提取多个字段的能力。这和针对字典的operator.itemgetter()使用很类似。

### 15 根据字段将记录分组
#### 15-1 问题
有一系列的字典或对象实例，我们想根据某个特定的字段(比如说日期)来分组迭代数据。
#### 15-2 解决方案
itertools.groupby()函数在对数据进行分组时特别有用。为了说明其用途，假设有如下的字典列表:

In [159]:
rows = [
    {'address':'5412 N CLARK','date':'07/01/2012'},
    {'address':'5148 N CLARK','date':'07/04/2012'},
    {'address':'5800 N 58TH','date':'07/02/2012'},
    {'address':'2122 N CLARK','date':'07/03/2012'},
    {'address':'5645 N RACENSWOOD','date':'07/02/2012'},
    {'address':'1060 N ADDISON','date':'07/02/2012'},
    {'address':'4801 N BROADWAY','date':'07/01/2012'},
    {'address':'1039 N GRANVI','date':'07/04/2012'},
    ]

现在假设想根据日期以分组的方式迭代数据。要做到这些，首先以目标字段(在这个例子中是date)来对序列排序，然后再使用itertools.groupby()。

In [161]:
from operator import itemgetter
from itertools import groupby
# sort by the desired field first
rows.sort(key=itemgetter('date'))
# Iterate in groups
for date, items in groupby(rows,key=itemgetter('date')):
    print(date)
    for i in items:
        print(' ',i)

07/01/2012
  {'address': '5412 N CLARK', 'date': '07/01/2012'}
  {'address': '4801 N BROADWAY', 'date': '07/01/2012'}
07/02/2012
  {'address': '5800 N 58TH', 'date': '07/02/2012'}
  {'address': '5645 N RACENSWOOD', 'date': '07/02/2012'}
  {'address': '1060 N ADDISON', 'date': '07/02/2012'}
07/03/2012
  {'address': '2122 N CLARK', 'date': '07/03/2012'}
07/04/2012
  {'address': '5148 N CLARK', 'date': '07/04/2012'}
  {'address': '1039 N GRANVI', 'date': '07/04/2012'}


#### 15-3 讨论
函数groupby()通过扫描序列找出拥有相同值(或是由参数key指定的函数所返回的值)的序列项，并将它们分组。groupby()创建了一个迭代器，而在每次迭代时都会返回一个值(value)和一个子迭代器(sub_iterator)，这个子迭代器可以产生所有在该分组内具有该值的项。
在这里重要的是首先要根据感兴趣的字段对数据进行排序。因为groupby()只能检查连续的项，不首先排序的话，将无法按所想的方式来对记录分组。
如果只是简单地根据日期将数据分组到一起，放进一个大的数据结构中以允许进行随机访问，那么利用defaultdict()构建一个一键多值字典(multidict)可能会更好。例如:

In [163]:
from collections import defaultdict
rows_by_date = defaultdict(list)
for row in rows:
    rows_by_date[row['date']].append(row)

这使我们可以方便的访问每个日期的记录，如下所示:

In [165]:
for r in rows_by_date['07/01/2012']:
    print(r)

{'address': '5412 N CLARK', 'date': '07/01/2012'}
{'address': '4801 N BROADWAY', 'date': '07/01/2012'}


对于后面这个例子，我们并不需要先对记录做排序。因此，如果不考虑内存方面的因素，这种方式回避先排序再用groupby()迭代要来的更快。

### 16 筛选序列中的元素
#### 16-1 问题
序列中含有一些数据，我们需要提取出其中的值或根据某些标准对序列做删减。
#### 16-2 解决方案
要删选序列中的数据，同上最简单的方法是使用列表推导式。例如:

In [166]:
mylist = [1,4,-5,-10,-7,2,3,-1]
[n for n in mylist if n>0]

[1, 4, 2, 3]

使用列表推导式的一个潜在缺点是如果原始输入非常大的话，这么做可能会产生一个庞大的结果。如果这是你需要考虑的问题，那么可以使用生成器表达式通过迭代的方式产生筛选的结果。例如:

In [167]:
pos = (n for n in mylist if n>0)
pos

<generator object <genexpr> at 0x10ca46c50>

有时候筛选的标准没法简单地表示在列表推导式或生成器表达式中。比如，假设筛选过程中涉及异常处理或者其他一些复杂的细节。基于此，可以将处理筛选逻辑的代码放到单独的函数中，然后使用內建的函数filter()处理。示例如下:

In [168]:
values = ['1','2','-3','-','4','N/A','5']

def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False
    
ivals = list(filter(is_int,values))
print(ivals)

['1', '2', '-3', '4', '5']


filter()创建了一个迭代器，因此如果我们想要的是列表形式的结果，请确保加上了list()

#### 16-3 讨论
列表推导式和生成器表达式通常是用来筛选数据的最简单和最直接的方式。此外，它们也具有同时对数据做转换的能力。例如:

In [169]:
mylist = [1,4,-5,10,-7,2,3,-1]
import math
[math.sqrt(n) for n in mylist if n >0]

[1.0, 2.0, 3.1622776601683795, 1.4142135623730951, 1.7320508075688772]

关于筛选数据，有一种情况是用新值替换掉不满足标准的值，而不是丢弃它们。例如，除了要找到正整数之外，我们也许还希望在指定的范围内将不满足的值替换掉。通常这可以通过将筛选条件移到一个条件表达式中来轻松实现:

In [174]:
clip_neg = [n if n >0 else 0 for n in mylist]
clip_neg

[1, 4, 0, 10, 0, 2, 3, 0]

另外值得一提的筛选工具是itertools.compress()，它接受一个可迭代对象以及一个布尔选择器序列作为输入。输出时，它会给出所有在相应的布尔仙则其中为True的可迭代对象。如果想把对一个序列的筛选结果施加到另一个相关的序列上，这就会非常有用:

In [177]:
address = [
    '5412 N CLARK',
    '5148 N CLARK',
    '5800 E 58TH',
    '2122 N CLARK',
    '5645 N RAVENSWOOD',
    '1060 W ADDISON',
    '4801 N BROADWAY',
    '1039 W GRANVILLE',
    ]

counts = [0,3,10,4,1,7,6,1]

我们想要构建一个地址列表，其中相应的count值要大于5。下面是我们可以尝试的方法:

In [178]:
from itertools import compress
more5 = [n>5 for n in counts]

In [179]:
list(compress(address,more5))

['5800 E 58TH', '1060 W ADDISON', '4801 N BROADWAY']

这里的关键在于首先创建一个布尔序列，用来表示哪个元素可满足我们的条件。然后compress()函数挑选出满足布尔值为True的相应的元素。
同filter()函数一样，正常情况下compress()会返回一个迭代器。因此，如果需要的话，得使用list()将结果转化为列表。

### 17 从字典中提取子集
#### 17-1 问题
我们想创建一个字典，其本身是另一个字典的子集。
#### 17-2 解决方案
利用字典推导式(dictionary comprehension)可轻松解决。例如:

In [182]:
prices = {
    'ACME':45.23,
    'AAPL':612.78,
    'IBM':205.55,
    'HPQ':37.20,
    'FB':10.75
    }
# make a dictionary of all prices over 200
p1 = {key:value for key,value in prices.items() if value>200}
p1

{'AAPL': 612.78, 'IBM': 205.55}

In [184]:
# make a dictionary of tech stocks
tech_names = {'AAPL','IBM','HPQ','MSFT'}
p2 = {key:value for key,value in prices.items() if key in tech_names}
p2

{'AAPL': 612.78, 'HPQ': 37.2, 'IBM': 205.55}

#### 17-3 讨论
大部分可以用字典推导式解决的问题也可以通过创建元组序列然后将它们传给dict()函数来完成。例如:

In [185]:
p1 = dict((key,value) for key,value in prices.items() if value >200)
p1

{'AAPL': 612.78, 'IBM': 205.55}

但是字典推导式的方案更清晰，而且实际运行起来也要快很多(以本例的字典prices来测试，效率要高2倍多)
有时候会有多种方法来完成同一件事情。例如，第二个例子可以重写:

In [186]:
# make a dictionary of tech stocks
tech_names = {'AAPL','IBM','HPQ','MSFT'}
p2 = {key:prices[key] for key in prices.keys() & tech_names}
p2

{'AAPL': 612.78, 'HPQ': 37.2, 'IBM': 205.55}

但是这种解决方案几乎要比第一种慢1.6倍。

### 18 将名称映射到序列的元素中
#### 18-1 问题
我们的代码时通过位置(即索引，下标)来访问列表或元组的，但有时候这会使代码变得有些难以阅读。我们希望可以通过名称来访问元素，以此来减少结构中对位置的依赖性。
#### 18-2 解决方案
相比普通的元组，collections.namedtuple()(命名元组)只增加了极小的开销就提供了这些便利。实际上collections.namedtuple()是一个工厂方法，它返回的是Python中标准元组类型的子类。我们提供给它一个类型名称以及相应的字段，它就返回一个可实例化的类，为你已经定义好的字段传入值等：

In [187]:
from collections import namedtuple
Subscriber = namedtuple('Subscriber',['addr','joined'])
sub = Subscriber('jonesy@example.com','2012-10-19')
sub

Subscriber(addr='jonesy@example.com', joined='2012-10-19')

尽管namedtuple的实例看起来就像一个普通的类实例，但它的实例与普通的元组是可互换的，而且支持所有普通元组